In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

def fsolve2(func,x0):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res=fsolve(func, x0)[0]
    return res if res!=x0 else np.NaN

In [2]:
# path='/project/s1174/lyan/code/projectData/NST_e-scattering/pion2pt_more/cA2.09.48.h5'
# with h5py.File(path) as f:
#     imom=0
#     c2pt_piC_dat=f['data/piC'][:,:,imom]
#     c2pt_pi0_dat=np.complex128(f['data/pi0_conn'][:,:,imom]+f['data/pi0_disc'][:,:,imom])
#     vev_dat=np.complex128(f['data/vev'][:])
#     if True:
#         func=lambda t:(t+np.roll(np.flip(t,axis=1),1,axis=1))/2
#         c2pt_piC_dat=func(c2pt_piC_dat)
#         c2pt_pi0_dat=func(c2pt_pi0_dat)

path='/project/s1174/lyan/code/projectData/NST_f/cA2.09.48/data_merge/NST_f_meson2pt.h5'
with h5py.File(path) as f:
    imom=0
    c2pt_piC_dat=f['diags/P/data/pi+_pi+'][:,:,imom]
    c2pt_pi0_dat=f['diags/P/data/pi0_pi0'][:,:,imom]+f['diags/pi0f-pi0i/data/pi0_pi0'][:,:,imom]
    vev_dat=f['VEV/pi0f/data/pi0'][:]
    if True:
        func=lambda t:(t+np.roll(np.flip(t,axis=1),1,axis=1))/2
        c2pt_piC_dat=func(c2pt_piC_dat)
        c2pt_pi0_dat=func(c2pt_pi0_dat)

enss=['a']
# enss=['a','b']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

In [ ]:
ens='a'
tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(ens2N_T[ens]-t))
tau=1
def C2pt2meff(C2pt):
    t=C2pt # dim=time
    t=(np.roll(t,-tau,axis=0))/(t)
    t=np.array([fsolve2(lambda m:((tfunc_C(m,xt+tau))/(tfunc_C(m,xt)))-ele, 0.0593) for xt,ele in enumerate(t)])
    return t

ds=[1,2,4,8,16,32]
# ds=[1,20]
cfgss=[(0,1200),(0,300),(300,600),(600,900),(900,1200)]
# cfgss=[(0,1228)]

fig, axs = yu.getFigAxs(len(ds),len(cfgss),scale=1,sharex=True,sharey=True)
yu.addRowHeader(axs,[f'd={d}' for d in ds],fontsize=30)
yu.addColHeader(axs,[f'N={cfgs[1]-cfgs[0]}: from {cfgs[0]} to {cfgs[1]}' for cfgs in cfgss],fontsize=30)
xunit=ens2a[ens]; yunit=ens2aInv[ens]


for i_d,d in enumerate(ds):
    for i_cfgs,cfgs in enumerate(cfgss):
        print(d,cfgs,end='              \r')
        ax=axs[i_d,i_cfgs]
        
        imin_cfg,imax_cfg=cfgs
        
        c2pt_piC_jk=yu.jackknife(np.real(c2pt_piC_dat[imin_cfg:imax_cfg]),d=d)
        c2pt_pi0_jk=yu.jackknife(np.real(c2pt_pi0_dat[imin_cfg:imax_cfg]),d=d)
        vev_jk=yu.jackknife(np.real(vev_dat[imin_cfg:imax_cfg]),d=d)
        c2pt_pi0_jk= c2pt_pi0_jk - vev_jk[:,None]**2
        
        meff_jk=yu.jackmap(C2pt2meff,c2pt_piC_jk)
        tmean,terr=yu.jackme(meff_jk)
        plt_x=range(len(tmean)); plt_y=tmean*yunit; plt_yerr=terr*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color='r')
        
        meff_jk=yu.jackmap(C2pt2meff,c2pt_pi0_jk)
        tmean,terr=yu.jackme(meff_jk)
        plt_x=range(len(tmean)); plt_y=tmean*yunit; plt_yerr=terr*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color='b')
        
        ax.set_ylim([0,300])

plt.tight_layout()
plt.savefig('fig/pion2pt_splitCfgs_deld.pdf')
plt.close()   
